In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import  pandas as pd
import numpy as np
import scipy.stats as stats

%matplotlib inline
from matplotlib import pyplot as plt

from scipy.stats import shapiro
from scipy import stats


In [ ]:
sns.set_theme(style="ticks", palette="pastel")

In [ ]:
path = '/kaggle/input/mobile-games-ab-testing/cookie_cats.csv'

cats_data = pd.read_csv(path)
cats_data.head(5)

In [ ]:
# 1. Process and clean data => Detect dupplicates => There aren't dupplicates

print("Total users:", cats_data['userid'].count())
print("Total different users':", cats_data['userid'].nunique())

In [ ]:
#The set is good for testing because thevnumber of players is more or less the same in the control and test group

graf_version=sns.countplot(x='version' ,data=cats_data)
graf_version.set(xlabel="version", ylabel="Total players",title='Total players by version')

In [ ]:
# Detect outliers => There is just an outlier (with almost 50.000 rounds) 

cats_data.describe([0.05, 0.10, 0.25, 0.75, 0.90, 0.95])[["sum_gamerounds"]]

#Mean is 51 rounds and 25% of players have <5 rounds during the first 7 days

In [ ]:
#Remove outliers
cats_data = cats_data[cats_data.sum_gamerounds < cats_data.sum_gamerounds.max()]

In [ ]:
#Almost 4000 users played no round in first 7d (Perhaps the game is too difficult for them??)
print('Total users with 0 rounds: ', cats_data[cats_data['sum_gamerounds'] == 0].userid.count())

In [ ]:
#The column version, the control group will have be 'A' and test group will be 'B'
cats_data["version"] = np.where(cats_data.version == "gate_30", "A", "B")

In [ ]:
#Create a column Retention = 1 when user play during the first 7 days after instalation, and 0 in other cases
#in control group (gate_30), 17% of users 
cats_data["Retention"] = np.where((cats_data.retention_1 == True) & (cats_data.retention_7 == True), 1,0)
cats_data.groupby(["version", "Retention"])["sum_gamerounds"].agg(["count", "median", "mean", "std", "max"])

In [ ]:
sns.countplot(x='version' ,data=cats_data)
#retention in group control (gate 30) is 14,9% and retention in test group is 14,3%.
#players who stay playing first 7 days have a median of 127 round in group A and 133 rounds in group B. 

In [ ]:
######################################################
#  A/B TESTING
#  First review with a Q-Q plit

#A/B test => 1-Split data in group A and group B
groupA = cats_data[cats_data['version'] == "A"]["sum_gamerounds"]
groupB = cats_data[cats_data['version'] == "B"]["sum_gamerounds"]
    
fig = plt.figure()
ax = fig.add_subplot()
res = stats.probplot(groupA, plot=plt)
#conclusion: The distribution is not normal

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
res = stats.probplot(groupB, plot=plt)
#conclusion: The distribution is not normal

In [ ]:
# 2-Shapiro–Wilk test for normality (α = 0.05 threshold) => No normal distribution
normalA = shapiro(groupA)[1] < 0.05  
normalB = shapiro(groupB)[1] < 0.05

print("H0 hypothesis: Distribution is Normal = False")
print("H1 hypothesis: Distribution is not Normal = True")
print(f'As the test is groupA is {normalA}, we rejected H0 hypotesis => Distribution A is not normal')
print(f'As the test is groupB is {normalB}, we rejected H0 hypotesis => Distribution B is not normal')


In [ ]:
# 3-Use Mann-Whitney U test (non-parametric test)
ttest = stats.mannwhitneyu(groupA, groupB)[1] 

print(f'The ttest is: {ttest}, so we rejected H0 hypotesis => Difference between A and B groups is statistically significant')

#So, there is a statistically significant difference between two groups.
#If we want to keep retention rate high — we should not move the gate from level 30 to level 40.